# Ejemplo NMF Rank-2 con dataset Reuters

Autor: Benjamin Ayancán, PUC Chile
Tutor: Denis Parra, PUC Chile

En el siguiente notebook desarrollaremos un ejemplo de cómo poder generar una descomposición jerarquica a través del algoritmo de rango bajo NMF rank-2.

* Para este tutorial usaremos el dataset de noticias Reuters que nos entrega la libería `keras`

* Los aspectos teoricos y detalles del algoritmo son abordados en el  siguiente [notebook de Observablehq](https://observablehq.com/@beayancan/descomposicion-jerarquica)

* Para hacer más pedagógico el ejemplo se va a reducir tanto la cantidad de datos (filas) como la cantidad de features (columnas)

---

## Configuración

### Notebooks

--> Si lo estás ejecutando en un `jupyter notebook` debes instalar las siguientes librerías

```py
!pip install tensorflow
!pip install keras
```

--> Si lo estás ejecutando en `colab`



### Librerías

In [1]:
# Chequeamos que nuestro soporte no tenga problemas

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [2]:
# importamos las liberías que vamos a utilizar
# herramientas y el dataset

import os, sys
import keras
import statistics
import collections
import numpy as np
from keras.datasets import reuters

Using TensorFlow backend.


---

## Reuters

#### ¿Por qué usar esta librería?

Reuters es un repositorio de documentos que podemos utilizar a través de `keras` o bien descargarlo en su [página web](http://konect.cc/networks/gottron-reuters/). Esta posee las siguientes características

* Son un conjunto de 11.228 noticias, etiquetados en 46 tópicos.


* Está preprocesado según un ranking de palabras de un volabulario, es decir su representación es a través de las posiciones en el ranking de las palabras de un documento
* Posee funciones sencillas que nos ayudan a manejar cómo representar los documentos sin gastar mucho tiempo en su preprocesamiento
* Se enfoca en el contenido de un documento, las palabras que lo representan, en vez del mensaje que este entregue

---

#### Cargar el dataset

In [3]:
# PARAMETROS
# https://www.tensorflow.org/api_docs/python/tf/keras/datasets/reuters/load_data

# Cantidad de palabras significativas del vocabulario a usar

# num_words = None # incluirlas todas
num_words = 100 # solo tomaremos las 100 primeras palabras
skip_top = 0 # palabras que no consideraremos

# máximo N° de palabras para representar a un documento
max_len = None # todas
# max_len = 50

# Porcentaje de palabras para usar en el test set
test_split = 0.025 # train set: 97.5%, test set: 2.5%

In [4]:
# Cargamos los de datos de clasificación de noticias de Reuters

(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=num_words,
                                                         maxlen=max_len,
                                                         test_split=test_split,
                                                         skip_top=skip_top)

In [5]:
# Revisamos cómo fueron cargados los datos

# y_train, y_test contendrán los labels de las clases

num_clases = max(y_train) + 1

print(f'Tamaño train set: {x_train.shape}')
print(f'Tamaño test set: {x_test.shape}')
print(f'Cantidad clases: {num_clases}')
print('Palabra mayor', min(len(e) for e in x_train))
print('Oalabra menor', max(len(e) for e in x_train))

Tamaño train set: (10947,)
Tamaño test set: (281,)
Cantidad clases: 46
Palabra mayor 2
Oalabra menor 2376


### Qué es lo que contienen
* Cada elemento del set contiene los índices de las palabras más utilizadas por un documento

* Las palabras están ordenadas según su frecuencia de aparición
* Además cada documento pertenece a un tópico en especifico
* Notar por el `x_train.shape` que las representaciones no tienen un largo especifico

In [6]:
# Mostremos algunos ejemplos
for i in range(1,15, 3):
  print(f'Doc: {i},  largo: {len(x_test[i])}, clase {y_test[i]}')
  print(f'   contenido: {x_test[i]} \n')

Doc: 1,  largo: 88, clase 20
   contenido: [1, 4, 2, 2, 2, 5, 2, 28, 4, 2, 37, 2, 2, 2, 2, 2, 6, 72, 2, 20, 22, 72, 2, 20, 54, 2, 2, 28, 4, 2, 2, 96, 5, 2, 2, 2, 62, 2, 2, 2, 20, 5, 4, 2, 2, 13, 9, 2, 2, 24, 96, 62, 2, 7, 2, 4, 60, 5, 2, 8, 10, 2, 5, 2, 11, 2, 5, 2, 2, 34, 2, 13, 2, 52, 2, 54, 2, 4, 2, 2, 5, 2, 34, 72, 2, 20, 17, 12] 

Doc: 4,  largo: 85, clase 4
   contenido: [1, 2, 2, 2, 2, 2, 5, 2, 2, 2, 2, 8, 36, 23, 2, 6, 2, 2, 18, 2, 79, 5, 2, 2, 71, 28, 2, 15, 2, 36, 8, 4, 2, 2, 5, 4, 2, 2, 79, 36, 2, 2, 41, 2, 2, 2, 44, 20, 5, 2, 2, 2, 2, 79, 2, 2, 8, 36, 91, 2, 95, 9, 2, 2, 2, 79, 36, 8, 4, 2, 9, 2, 2, 33, 2, 28, 2, 2, 2, 2, 94, 2, 89, 17, 12] 

Doc: 7,  largo: 49, clase 3
   contenido: [1, 53, 2, 26, 14, 2, 26, 39, 32, 2, 18, 14, 12, 2, 18, 88, 2, 19, 11, 14, 2, 72, 11, 2, 2, 53, 2, 26, 14, 2, 26, 39, 44, 2, 18, 14, 44, 2, 18, 88, 2, 59, 11, 14, 2, 63, 11, 17, 12] 

Doc: 10,  largo: 286, clase 16
   contenido: [1, 4, 2, 2, 2, 2, 2, 2, 28, 2, 7, 2, 40, 85, 2, 2, 2, 4, 2, 9, 2,

In [7]:
mapping = ['cocoa','grain','veg-oil','earn','acq','wheat','copper','housing','money-supply',
           'coffee','sugar','trade','reserves','ship','cotton','carcass','crude','nat-gas',
           'cpi','money-fx','interest','gnp','meal-feed','alum','oilseed','gold','tin',
           'strategic-metal','livestock','retail','ipi','iron-steel','rubber','heat','jobs',
           'lei','bop','zinc','orange','pet-chem','dlr','gas','silver','wpi','hog','lead']

train_count = collections.Counter(y_train)
test_count = collections.Counter(y_test)
total_words = [statistics.mean([len(e) for e in x_train[y_train.flatten() == i]]) for i in range(46)]

print("         Las clases y sus estadísticas")
print("{:4s} {:20s} {:5s}  {:5s} {:7s}".format("Idx","Clase", "train", "test", "Palabras"))
for i in range(46):
    print("{:5d} {:20s} {:5d} {:5d}   {:6.2f}".format(i,mapping[i], train_count[i], test_count[i], total_words[i]))

         Las clases y sus estadísticas
Idx  Clase                train  test  Palabras
    0 cocoa                   64     3   236.69
    1 grain                  526    11   187.91
    2 veg-oil                 91     3   180.86
    3 earn                  3867   105    87.05
    4 acq                   2373    50   136.51
    5 wheat                   22     0   188.41
    6 copper                  61     1   154.18
    7 housing                 19     0   166.26
    8 money-supply           170     7   192.82
    9 coffee                 125     1   215.36
   10 sugar                  149     5   186.44
   11 trade                  461    12   256.77
   12 reserves                61     1   184.51
   13 ship                   202     7   166.66
   14 cotton                  28     0   143.11
   15 carcass                 28     1   193.11
   16 crude                  530    13   223.52
   17 nat-gas                 49     2   161.08
   18 cpi                     80     6   164.34
 

### Qué es lo que representan
* Solo tenemos los índices, pero ¿de qué?
  * Son la posición rankeada de la palabra
* Usamos la indexación del vocabulario del dataset para identificar la palabra

In [8]:
# Tenemos el vocabulario con las palabras y sus indices
word_index = reuters.get_word_index(path="reuters_word_index.json")

# Generamos un diccionario con el contenido
index_to_word = { value+3 : key for key, value in word_index.items() }

# Indices reservados
index_to_word[0] = '-PAD-'   # 0: carpeta
index_to_word[1] = '-START-' # 1: inicio secuencia
index_to_word[2] = '-UNK-'   # 2: elemento no encontrado

len_index = len(index_to_word)

## Preprocesamiento

* Para poder realizar los calculos, necesitamos
  * Una estructura regular de datos
  * Que cada documento posea un mismo largo

* Pasaremos primero los datos a una menor dimensión reduciendo las palabras y eliminando lo innecesario

In [9]:
# Primero eliminaremos las referencias a elementos que no se encuentran
# dentro de las 1000 palabras más usadas en el vocabulario
# ademas de las entradas reservadas

  # Eliminamos
  # 0: '-PAD-'
  # 1: '-START-'
  # 2: '-UNK-'
  # 12: '3'
  # 17: 'reuter'

def filtrar_relevante(arreglo, por_eliminar=[0,1,2]):
  """
  Borra las palabras que pertenecen a los indices del array por_eliminar
  """
  return list(filter(lambda x: x not in por_eliminar, arreglo))

def eliminar_reservadas(x_array):
  """
  Eliminamos las entradas reservadas y entradas inutiles
  retorna el contenido homogeneo del doc
  """
  por_eliminar = [0,1,2,12,17]
  largo_test, = x_array.shape
  for i in range(largo_test):
    x_array[i] = filtrar_relevante(x_array[i], por_eliminar)
  return x_array

In [10]:
# Para hacer un ejemplo más sencillo de entender
# Selecionaremos solo las primeras 7 clases

def reducir_labels(data_array, labels, k=7):
  """
  Filtramos los documentos que pertenezcan a las primeras k clases
  Retornamos el arreglo con los documentos y sus correspondientes labels
  """

  retorno, retorno_labels = list(), list()
  for i in range(len(data_array)):
    if labels[i] < k:
      retorno.append(data_array[i])
      retorno_labels.append(labels[i])
  return np.array(retorno), retorno_labels

In [11]:
x_test = eliminar_reservadas(x_test)
x_data, y_data = reducir_labels(x_test, y_test, k=7)
print(x_data.shape)

(173,)


In [12]:
# Mostremos lo que contiene
for i in range(1,15, 3):
  print(f'Doc: {i},  Largo: {len(x_data[i])}, Clase {y_data[i]}')
  print(f' {x_data[i]}', '\n', ' '.join([index_to_word[word] for word in x_data[i]]), "\n")

Doc: 1,  Largo: 55, Clase 3
 [71, 8, 25, 10, 5, 68, 13, 67, 5, 4, 80, 6, 5, 93, 16, 8, 4, 6, 68, 5, 10, 67, 5, 80, 5, 28, 42, 96, 5, 15, 8, 4, 91, 30, 92, 83, 4, 5, 58, 5, 10, 13, 28, 20, 5, 4, 80, 4, 49, 24, 16, 40, 6, 23, 10] 
 inc said its a of one for share of the stock to of april it said the to one of a share of stock of at an price of dlrs said the may be after 10 the of or of a for at pct of the stock the company that it has to is a 

Doc: 4,  Largo: 71, Clase 3
 [53, 19, 15, 14, 26, 39, 59, 11, 14, 32, 11, 86, 19, 35, 14, 19, 32, 35, 53, 15, 14, 32, 15, 39, 11, 14, 70, 11, 86, 63, 35, 14, 44, 35, 89, 9, 68, 92, 5, 11, 15, 58, 26, 10, 67, 13, 7, 4, 98, 5, 42, 7, 48, 39, 27, 26, 10, 67, 13, 7, 21, 89, 34, 72, 35, 15, 50, 49, 8, 34, 10] 
 shr 1 dlrs vs cts net 6 mln vs 2 mln revs 1 billion vs 1 2 billion shr dlrs vs 2 dlrs net mln vs 0 mln revs 8 billion vs 5 billion 1987 and one after of mln dlrs or cts a share for in the quarter of an in 1986 net by cts a share for in on 1987 w

### Pasar los datos a matriz doc-term

* Pasamos a una matriz $A \in \mathbb{R}^{m \times n}$ donde $m$ es la cantidad de documentos y $n$ es la cantidad de palabras del vocabulario
* Pasaremos los datos a una representación de $\{0, 1\}$
* Representando la entrada $A[i,j]$ la aparición en el $i$-ésimo documento la $j$-ésima palabra

In [13]:
# Importamos tokenizer para producirlo

from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

tkn = Tokenizer(num_words=num_words) # tamaño del vocabulario
num_clases = max(y_data) + 1

In [14]:
# Contruimos la matriz pasando los datos a binario

x_data_bin = tkn.sequences_to_matrix(x_data, mode='binary')
y_data_cat = to_categorical(y_data, num_clases)

In [15]:
# Revisamos los como queda la representacion

entry = 10
print(f'Train Set: {x_data_bin.shape} \n {x_data_bin[entry]}\n')
print(f'Test Set: {y_data_cat.shape}   (usaremos este) \n {y_data_cat[entry]}')

Train Set: (173, 100) 
 [0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 1.]

Test Set: (173, 7)   (usaremos este) 
 [0. 0. 0. 1. 0. 0. 0.]


In [16]:
# Siguiendo la interpretación del paper

A_matrix = np.transpose(x_data_bin)

# NMF Jerárquico

* Ya tenemos unos datos preprocesados de forma ideal para nuestro trabajo
* Realizaremos la secuencia de NMF jerárquico siguiendo el paper
  * [Fast Rank-2 Nonnegative Matrix Factorization for Hierarchical Document Clustering](https://smallk.github.io/papers/hierNMF2.pdf)

* Recordar que el objetivo es minimizar la siguiente operación
  $$\min_{W \geq 0, H \geq 0} ||A - WH||_2^{2}$$
  * A través de la resolución de los subproblemas convexos
    $$\min_{H \geq 0} ||A - WH||_2^{2}$$
    $$\min_{W \geq 0} ||A^T - H^T W^T||_2^{2}$$

## NMF Rank-2

* Usaremos el algoritmo Rank-2 para generar una estructura de árbol binario jerárquico 

In [17]:
def rank2(A, W):
  """
  Recibe las matrices objetivo A y su matriz izquierda W
  Calcula la resolución iterativa de la minimización según el paper
  y obtenemos la minimización de H a partir de W
  Retorna las matrices W, H de las descomposición
  """
  m, n = np.shape(A)

  # resolvemos por minimos cuadrados
  H = np.linalg.solve(np.dot(np.transpose(W), W), np.dot(np.transpose(W), A))

  # Separamos en columnas
  w1, w2 = W[:, 0], W[:, 1]
  beta1, beta2 = np.linalg.norm(w1), np.linalg.norm(w2)

  # normalizamos
  u, v = np.dot(np.transpose(A), w1)/beta1, np.dot(np.transpose(A), w2)/beta2

  for j in range(n):
    # Para cada vector determinamos si cumple con la solucion
    retorno_j = np.zeros(2)
    if (H[:, j] >= 0).all():
      continue
    elif u[j]*beta1 >= v[j]*beta2:
      retorno_j[0] = u[j]
    else:
      retorno_j[1] = v[j]
    H[:, j] = retorno_j
  return W, H

def NMF_rank2(A, W=None, H=None, k=2, **kwargs):
  """
  Recibe la matriz objetivo y matrices iniciales
  Se realiza dos veces la minimización primero para H
  y luego para W
  Retorna la descomposición W, H de baja calidad
  """
  m, n = np.shape(A)

  # Iniciamos las matrices
  if W is None:
    W = np.random.rand(m, k)

  if H is None:
    H = np.zeros((k, n))
  
  # Realizamos las minimizaciones
  W, H = rank2(A, W)
  HT, WT = rank2(np.transpose(A), np.transpose(H))
  # Retornamos los valores que resultaron minimizados
  return np.transpose(WT), np.transpose(HT)

In [18]:
def calculo_NMF(A, max_iteraciones=15, k=2, W=None, H=None, error=0.5):
  """
  Recibe la matriz objetivo A, dimension k,
  máximo de iteraciones y matrices iniciales
  Realiza de forma recursiva la aplicación de rank-2
  para así obtener una mejor aproximación
  Retorna los elementos W, H que aproximan A
  al alcanzar una cota de error o superar el maximo
  """
  # Inicializamos las matrices
  m, n = np.shape(A)
  if W is None:
    W = np.random.rand(m, k)

  for i in range(max_iteraciones):
    W, H = rank2(A, W)
    HT, WT = rank2(np.transpose(A), np.transpose(H))
    W, H = np.transpose(WT), np.transpose(HT)
    if (np.linalg.norm(A - np.dot(W, H))) < error: break
  return W, H

In [19]:
def normalizar_descomposicion(W, H):
  """
  Normaliza las columnas de W y pondera respectivamente
  las filas de H para el resultado esperado
  """

  for j in range(2):
    norma = np.linalg.norm(W[:, j])
    W[:, j] = W[:, j]/norma
    H[j, :] = H[j, :]*norma
  return W, H


def calcular_descomposicion(A_matrix, max_iteraciones=15, max_intentos=10):
  """
  Recibe matriz objetivo, cantidad maxima iteraciones e intentos de calcular
  Calcula la descomposición rank-2 de forma reiterativa
  Si el i-esimo intento alcanza la cota
  se retorna la descomposición W, H
  """
  salida, excepcion = False, False
  for i in range(max_intentos):
    try:
      W, H = calculo_NMF(A_matrix, max_iteraciones, k=2, W=None, H=None)
      error = np.linalg.norm(np.dot(W, H) - A_matrix)
      if error < 60:
        salida = True
    except:
      excepcion = True
    else:
      if not excepcion and salida:
        return normalizar_descomposicion(W, H)

### Estructura jerárquica

* Para ir generando una estructura jerárquica debemos poder determinar cómo hacer split de los datos

  * Necesitamos determinar donde conviene más separar los datos
  * Para esto necesitamos una métrica
    * Utilizaremos la misma distribución de las palabras que entregan las columnas de la matriz $W$ de la descomposición

* Además debemos saber si el split que vamos a hacer conviene, pues deben ser operaciones optimas

* El dividir y conquistar los datos nos permitirá realizar el algoritmo de forma recursiva

  * Aplicaremos NMF haremos split de los datos
  * A estos dos hijos de datos les aplicaremos NMF
  * Continuaremos hasta alcanzar cierto objetivo


In [20]:
def idx_plbs(arreglo):
  """
  Recibe un arreglo de palabras
  Genera un diccionario con los detalles de la palabra
  retornando una lista ordenada según relevancia
  """
  largo = len(arreglo)
  retorno = list({'word': i, 'value': arreglo[i]} for i in range(largo))
  retorno = sorted(retorno, key=lambda x: x['value'], reverse=True)

  for i in range(largo):
    retorno[i]['id'] = i
  return retorno

def generar_arrays(array_N, array_L, array_R):
  """
  Recibe los arreglos para poder dividir
  Retorna los arreglos ordenados según relevancia de sus palabras
  """
  return idx_plbs(array_N), idx_plbs(array_L), idx_plbs(array_R)

In [21]:
def factor_descuento(word, array_L, array_R):
  """
  Recibe los arreglos y la palabra para la cual
  se va a calcular su descuento
  Retorna el descuento de la palabra
  """

  fi_L = next(x for x in array_L if x['word'] == word)
  fi_R = next(x for x in array_R if x['word'] == word)  
  return np.log2(len(array_L) - max(fi_L['id'], fi_R['id']) + 1)


def ganancia_palabra(word, array_N, array_L, array_R):
  """
  Recibe los arreglos y la palabra de la que se quiere obtener su ganancia
  Retorna la ganancia de la palabra
  """
  
  descuento = factor_descuento(word, array_L, array_R)
  elemento = next(x for x in array_N if x['word'] == word)
  return np.log2(len(array_L) - elemento['id'] + 1)/descuento


def ganancias(array_N, array_L, array_R):
  """
  Calcula la ganancia del arreglo
  Retorna el arreglo de las ganancias y ordenada según ganancia
  """
  retorno = list()
  for word in range(len(array_N)):
    gan_actual = ganancia_palabra(word, array_N, array_L, array_R)
    retorno.append({'palabra': word, 'ganancia': gan_actual})
  
  return retorno, sorted(retorno, key=lambda x: x['ganancia'], reverse=True)

In [22]:
def MDCG(gan_array):
  """
  Calculo de MDCG según el array que se entregue
  Retorna el valor de ganancia
  """
  largo = len(gan_array)
  elementos = list(gan_array[i]['ganancia']/np.log2(i+1) for i in range(1, largo))
  return gan_array[0]['ganancia'] + sum(elementos)


def mNDCG(gan_array, gan_sort):
  """
  Calculo del puntaje a través de los arrays listos
  """
  return MDCG(gan_array)/MDCG(gan_sort)


def puntaje(f_N, f_L, f_R):
  """
  Calcula el puntaje de la descomposición NMF actual
  Retorna el valor que nos ayuda a decidir
  """
  gan, gan_sort = ganancias(*generar_arrays(f_N, f_L, f_R))
  return mNDCG(gan, gan_sort)**2

In [23]:
def elem_puntaje(A_matrix, L_matrix, R_matrix):
  """
  Calcula la descomposición NMF de A (nodo)
  y de sus posibles hijos
  Retorna los elementos necesarios para determinar si conviene
  """

  condicion = False
  while not condicion:
    try:
      W, H = calcular_descomposicion(A_matrix)
      WL, HL = calcular_descomposicion(L_matrix)
      WR, HR = calcular_descomposicion(R_matrix)
      condicion = True
    except:
      condicion = False
    else:
      if condicion:
        return W, H, WL, HL, WR, HR

def calculo_puntajes(W, H, WL, HL, WR, HR, i):  
  """
  Calcula el puntaje de los hijos del nodo
  a partir de los 
  """
  X = W[:, i].copy()

  puntaje_N1 = puntaje(X, WL[:, 0], WL[:, 1])
  puntaje_N2 = puntaje(X, WR[:, 0], WR[:, 1])

  return puntaje_N1, puntaje_N2


def puntajes_hijos(A, L, R):
  """
  Genera el calculo del puntaje a partir de los
  elementos necesario a partir del nodo
  """
  # W, H, WL, HL, WR, HR = elem_puntaje(A, L, R)
  return calculo_puntajes(*elem_puntaje(A, L, R), 0)
  

In [24]:
def agregar_columna(A, columna):
  """
  Agrega columna a la matriz A sin importar su contenido
  Retorna la matriz con la columna añadida
  """
  if A is None:
    A = np.zeros((len(columna), 1))
    A[:, 0] = columna
  else:
    A = np.column_stack((A,columna))
  return A


def split_matrix(A_matrix, W, H):
  """
  Separación de la matriz por contenido
  Retorna la separación en dos matrices
  """
  m, n = np.shape(A_matrix)

  A1, A2 = None, None

  for j in range(n):
    if H[0][j] > H [1][j]:
      A1 = agregar_columna(A1, A_matrix[:, j])
    else:
      A2 = agregar_columna(A2, A_matrix[:, j])

  if A1.shape[1] >= A2.shape[1]:
    return A1, A2
  return A2, A1


# Parte final

Vamos a generar un arreglo que contenga la estructura de nuestro arbol
* Será un ejemplo sencillo por lo que usaremos pocos nodos
* Usamos un arreglo para los nodos generado
* Retornaría este arreglo que describe la estructura

In [25]:
# Variables globales

numero_nodos = 7 # cantidad nodos para crear
beta = 1.1 # diferencia de tamaño mínima que habrá entre los nodos

In [26]:
def seleccionar_nodo(lista_nodos):
  """
  Recibe el arreglo de los nodos de la estructura
  Calcula cual nodo es conveniente separar y lo retorna
  """
  if len(lista_nodos) == 1:
    return lista_nodos[0]
  lista_nodos = sorted(lista_nodos,
                       key = lambda i: i['puntaje'],
                       reverse=True)
  return lista_nodos[1]

def menor_puntaje(lista_nodos, puntaje_N2):
  """
  Determina si el puntaje actual es el
  menor comparando con todos los nodos
  Retorna bool si conviene hacer split a ese nodo
  """
  for elemento in lista_nodos:
    if elemento['puntaje'] <= puntaje_N2:
      return False
  return True

In [27]:
def arreglo_palabras(W, i):
  """
  Genera el arreglo de las palabras de la columna i
  Retorna arreglo diccionarios con los datos ordenados
  """

  entradas = ['idx', 'value']
  distribucion, retorno = W[:, i], list()
  for item in enumerate(distribucion):
    retorno.append(dict(zip(entradas, item)))
  return sorted(retorno, key=lambda i: i['value'], reverse=True)


def encontrar_significado(arreglo):
  """
  Recibe el arreglo de indices de palabras
  Retorna los elementos con atributo word que es el significado
  """
  for i in range(len(arreglo)):
    arreglo[i]['word'] = index_to_word[arreglo[i]['idx'] + 4]
  return arreglo

In [28]:
def arreglo_palabras(W, i):
  entradas = ['idx', 'value']
  distribucion = W[:, i]
  retorno = list()
  for item in enumerate(distribucion):
    retorno.append(dict(zip(entradas, item)))
  retorno = sorted(retorno, key = lambda i: i['value'], reverse=True)
  return retorno


def encontrar_significado(arreglo):
  # Como teniamos ordenada la representacion
  # Usamos nuestro index to words para encontrar la palabra

  for i in range(len(arreglo)):
    arreglo[i]['word'] = index_to_word[arreglo[i]['idx'] + 4]
  return arreglo


def palabras_destacadas(W, cantidad=3):
  n = int(np.round(cantidad/2))+1
  arreglo_1 = encontrar_significado(arreglo_palabras(W, 0)[:n])
  arreglo_2 = encontrar_significado(arreglo_palabras(W, 1)[:n])
  arreglo_1.extend(arreglo_2)
  return arreglo_1
  


In [29]:
def jerarquizacion(A_matrix):
  """
  Genera la estructura de jerarquía realizando
  descomposiciones NMF de forma recursiva
  Retorna la estructura 
  """

  outliner = None
  lista_nodos = list()

  primer_nodo = {
    'id': 0,
    'parent': None,
    'matrix': A_matrix,
    'puntaje': 1,
    'shape': A_matrix.shape,
  }

  lista_nodos.append(primer_nodo)

  for i in range(1, numero_nodos, 2):
    M = seleccionar_nodo(lista_nodos)
    W, H = calcular_descomposicion(M['matrix'])
    M['W'], M['H'] = W, H
    N1, N2 = split_matrix(M['matrix'], W, H)
    puntaje_N1, puntaje_N2 = puntajes_hijos(M['matrix'], N1, N2)
    
    N1_nodo = {
    'id': i+1,
    'parent': M['id'],
    'matrix': N1,
    'puntaje': puntaje_N1,
    'shape': N1.shape,
    'hijos': None
    }

    N2_nodo = {
    'id': i+2,
    'parent': M['id'],
    'matrix': N2,
    'puntaje': puntaje_N2,
    'shape': N2.shape,
    'hijos': None
    }

    M['hijos'] = [i+1, i+2, ]

    lista_nodos.append(N1_nodo)
    lista_nodos.append(N2_nodo)

  return lista_nodos

In [30]:
def matriz_W_lista(lista_nodos):
  for nodo in lista_nodos:
    if 'W' not in nodo.keys():
      nodo['W'], nodo['H'] = calcular_descomposicion(nodo['matrix'])
  return lista_nodos


def limpiar_lista(lista_nodos):
  elementos = ['matrix', 'W', 'H']
  new_list = [{k: v for k, v in d.items() if k not in elementos} for d in lista_nodos]
  return new_list

In [31]:
lista_arbol = jerarquizacion(A_matrix)

In [32]:
for nodo in lista_arbol:
  if 'W' not in nodo.keys():
    nodo['W'], nodo['H'] = calcular_descomposicion(nodo['matrix'])
    
  retorno = palabras_destacadas(nodo['W'], cantidad=10)
  #for palabra in retorno:
  #  print(palabra['word'])
  #print("")
  nodo['destacadas'] = retorno

In [33]:
limpiar_lista(lista_arbol)

[{'id': 0,
  'parent': None,
  'puntaje': 1,
  'shape': (100, 173),
  'hijos': [2, 3],
  'destacadas': [{'idx': 14, 'value': 0.37536499106798366, 'word': '000'},
   {'idx': 53, 'value': 0.3471975061129189, 'word': 'have'},
   {'idx': 26, 'value': 0.3467574791463768, 'word': 'be'},
   {'idx': 39, 'value': 0.3270400970668038, 'word': 'as'},
   {'idx': 11, 'value': 0.2806841094608583, 'word': 'dlrs'},
   {'idx': 19, 'value': 0.2685838779716942, 'word': 'is'},
   {'idx': 4, 'value': 0.26769512649315785, 'word': 'said'},
   {'idx': 6, 'value': 0.2669035815617256, 'word': 'a'},
   {'idx': 8, 'value': 0.26416458915879476, 'word': '3'},
   {'idx': 10, 'value': 0.2557990964676358, 'word': 'vs'},
   {'idx': 5, 'value': 0.24497288891977692, 'word': 'and'},
   {'idx': 9, 'value': 0.23906860895636386, 'word': 'for'}]},
 {'id': 2,
  'parent': 0,
  'puntaje': 0.49908531073766027,
  'shape': (100, 99),
  'hijos': [4, 5],
  'destacadas': [{'idx': 27, 'value': 0.2388270897606621, 'word': 'with'},
   {'i

## TO DO

* Visualización del arbol en Observable

* Uso dataset Andrés Carvallo